# Islington Data Science Capstone

## Business Case

The business problem I am suggesting is that a friend wants to open a sushi restaurant in the Islington area of London. They want to know the most  popular restaurant food types already in the area (including competitive sushi restaurants) and also when different types of restaurants are popular so that they can target their marketing and resources accordingly.

## Data Requirements

The data that I will need is a list of the restaurants in the local area, the type of cuisine that they serve, the times that they are open and the times that they are popular. I'll include data on other types of restaurants so that my model learns about when restaurants are generally popular in the local area. Including data on other types of restaurants will help to prevent my model from over-fitting. It may also highlight a gap in the market if there are other restaurants that are open and popular but existing sushi restaurants are not open.